# Import thư viện

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import re
import time 

d:\TLCN\Source Code\GPT test\gptenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Url seed & keyword

In [2]:
url_skip = [
    "/",
    "/cong-dong",
    "/ho-so-suc-khoe",
    "/cam-nang-y-te",
    "/huong-dan-di-kham-1",
    "/san-phu-khoa-0",
    "/benh-xuong-khop-0",
    "/giao-duc-gioi-tinh-0",
    "/benh-tieu-hoa-0",
    "/tai-bien-mach-mau-nao-0",
    "/tu-dien-xet-nghiem-0",
    "/noi-khoa-0",
    "/vaccine-covid--0",
    "/covid-va-cham-soc-f--0",
    "/rang-mieng-0",
    "/covid--0",
    "/benh-tim-mach-0",
    "/nam-hoc-1",
    "/tai-mui-hong-0",
    "/mat-0",
    "/nhi-khoa-0",
    "/benh-truyen-nhiem-0",
    "/hau-mon-truc-trang-0",
    "/tin-tuc-0",
    "/suc-khoe-sinh-san-0",
    "/phuc-hoi-chuc-nang-0",
    "/tam-than-tam-ly-0",
    "/benh-ly-da-lieu-0",
    "/benh-noi-tiet-0",
    "/benh-than-kinh-0",
    "/than-tieu-nieu-0",
    "/tiem-chung-0",
    "/kham-tong-quat-0",
    "/ngo-doc-0",
    "/di-truyen-0",
    "/benh-ly-di-ung-0",
    "/bac-si-oi-nhan-ai-0",
    "/goi-kham-suc-khoe-0",
    "/benh-ly-giao-mua-0",
    "/huong-dan-su-dung-1",
    "/giu-suc-khoe-dip-le-0",
    "/y-hoc-co-truyen-0",
    "/sinh-ly-hoc-tham-do-chuc-nang-0",
    "/podcast-ivie-bac-si-oi-0",
    "/chan-doan-hinh-anh-0",
    "/benh-ho-hap-0",
    "/bac-si-noi-bat",
    "/tim-benh-vien",
    "/dich-vu",
    "/tro-thanh-nha-ban-hang-chien-luoc-cung-ivie-shopping-mall",
    "/cong-dong",
    "/cam-nang-y-te",
    "/doi-tac",
    "/dieu-khoan-su-dung-cua-isofhcare-0",
    "/chinh-sach-bao-mat-thong-tin-isofhcare-0",
    "/quy-che-hoat-dong-cua-website-san-giao-dich-thuong-mai-dien-tu-isofhcare-com-0"
]

url_seed = {
    "/san-phu-khoa-0": "Sản phụ khoa",
    "/benh-xuong-khop-0": "Bệnh xương khớp",
    "/giao-duc-gioi-tinh-0": "Giáo dục giới tính",
    "/benh-tieu-hoa-0": "Bệnh tiêu hóa",
    "/tai-bien-mach-mau-nao-0": "Tai biến mạch máu não",
    "/tu-dien-xet-nghiem-0": "Từ điển xét nghiệm",
    "/noi-khoa-0": "Nội khoa",
    "/vaccine-covid--0": "Vaccine COVID",
    "/covid-va-cham-soc-f--0": "COVID và chăm sóc F0",
    "/rang-mieng-0": "Răng miệng",
    "/covid--0": "COVID",
    "/benh-tim-mach-0": "Bệnh tim mạch",
    "/nam-hoc-1": "Nam học",
    "/tai-mui-hong-0": "Tai mũi họng",
    "/mat-0": "Mắt",
    "/nhi-khoa-0": "Nhi khoa",
    "/benh-truyen-nhiem-0": "Bệnh truyền nhiễm",
    "/hau-mon-truc-trang-0": "Hậu môn trực tràng",
    "/suc-khoe-sinh-san-0": "Sức khỏe sinh sản",
    "/phuc-hoi-chuc-nang-0": "Phục hồi chức năng",
    "/tam-than-tam-ly-0": "Tâm thần tâm lý",
    "/benh-ly-da-lieu-0": "Bệnh lý da liễu",
    "/benh-noi-tiet-0": "Bệnh nội tiết",
    "/benh-than-kinh-0": "Bệnh thần kinh",
    "/than-tieu-nieu-0": "Thận tiết niệu",
    "/tiem-chung-0": "Tiêm chủng",
    "/ngo-doc-0": "Ngộ độc",
    "/di-truyen-0": "Di truyền",
    "/benh-ly-di-ung-0": "Bệnh lý dị ứng",
    "/benh-ly-giao-mua-0": "Bệnh lý giao mùa",
    "/giu-suc-khoe-dip-le-0": "Giữ sức khỏe dịp lễ",
    "/y-hoc-co-truyen-0": "Y học cổ truyền",
    "/sinh-ly-hoc-tham-do-chuc-nang-0": "Sinh lý học thăm dò chức năng",
    "/chan-doan-hinh-anh-0": "Chẩn đoán hình ảnh",
    "/benh-ho-hap-0": "Bệnh hô hấp"
}

url_template = "https://ivie.vn{}"

# Crawling

In [19]:
data_dict = {
    "specialty": [], 
    "title": [],
    "text": []
}

In [20]:
def get_html(url):
    try_again = 0
    while True:
        try:
            # Gửi yêu cầu HTTP để lấy nội dung HTML
            response = requests.get(url)
            
            if response.status_code == 200:
                html_content = response.text
                if html_content is None:
                    try_again = 1
                    print(f"BeautifulSoup: None to {url}! Try again ({try_again}) in 3 seconds")
                    time.sleep(3)
                    continue
                return html_content
        except:
            if try_again>=20:
                return None
            try_again += 1
            print(f"BeautifulSoup: Unable to connect to {url}! Try again ({try_again}) in 3 seconds")
            time.sleep(3)
        
def get_html_selenium(url):
    try_again = 0
    while True:
        try:
            # Khởi động trình duyệt Chrome với WebDriverManager
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.set_page_load_timeout(10)
            # Điều hướng đến trang web
            driver.get(url)

            # Lấy mã HTML của trang
            html_source = driver.page_source
            driver.quit()
            return html_source
        except:
            if try_again==10:
                return None
            try_again += 1
            print(f"Selenium: Can't get connection to {url}! Try again ({try_again}) in 3 seconds.")
            time.sleep(3)

def save_csv(data):
    df = pd.DataFrame(data)
    df.to_csv("nhi_khoa_ivie_article_crawling.csv",index=False)

def crawling(url):
    href_processed = []
    try:
        page=1
        while True:
            url_page = url_template.format(url + "?page=" + str(page))
            html_content_page = get_html_selenium(url_page)
            if html_content_page is None:
                break
            soup = BeautifulSoup(html_content_page, 'html.parser')
            url_article = []
            links = soup.find_all('a')
            for link in links:
                href = link.get('href')
                if href is not None:
                    if href not in url_skip and "/" in href and url not in href and "/tim-kiem/" not in href and href not in href_processed and "http" not in href and "www" not in href:
                        url_article.append(url_template.format(href))
                        href_processed.append(href)
            if len(url_article) == 0:
                break
            for link in tqdm(url_article, desc=f"Phân tích khoa {url_seed[url]} trang {page}: "):
                html_content = get_html(link)
                if html_content is None:
                    continue
                soup = BeautifulSoup(html_content, 'html.parser')
                text = re.sub(r'\n+', '\n', soup.select('div.detail__DetailBody-sc-xxj5tu-4.styleHelpers__styledSystem-sc-1x1jqqv-0.hYPtGx.eKGzPD.mostly-customized-scrollbar.content-html')[0].text)
                specialty = url_seed[url]
                title = soup.find_all('h1')[0].text
                data_dict["specialty"].append(specialty)
                data_dict["title"].append(title)
                data_dict["text"].append(text)
            save_csv(data_dict)
            print("ĐÃ LƯU!")
            page+=1
    except:
        print(f"Error with {url_seed[url]}! Current page: {page}")

In [5]:
with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(crawling, [u for u in url_seed]))

Selenium: Can't get connection to https://ivie.vn/san-phu-khoa-0?page=1! Try again (1) in 3 seconds.


Phân tích khoa Bệnh xương khớp trang 1:  84%|████████▍ | 16/19 [00:11<00:03,  1.02s/it]

Selenium: Can't get connection to https://ivie.vn/benh-tieu-hoa-0?page=1! Try again (1) in 3 seconds.


Phân tích khoa Bệnh xương khớp trang 1: 100%|██████████| 19/19 [00:14<00:00,  1.34it/s]


ĐÃ LƯU!
Selenium: Can't get connection to https://ivie.vn/san-phu-khoa-0?page=1! Try again (2) in 3 seconds.


...

ĐÃ LƯU!


Phân tích khoa Bệnh hô hấp trang 7: 100%|██████████| 4/4 [00:02<00:00,  1.67it/s]


ĐÃ LƯU!
Selenium: Can't get connection to https://ivie.vn/chan-doan-hinh-anh-0?page=9! Try again (1) in 3 seconds.
Selenium: Can't get connection to https://ivie.vn/benh-ho-hap-0?page=8! Try again (1) in 3 seconds.
Selenium: Can't get connection to https://ivie.vn/chan-doan-hinh-anh-0?page=9! Try again (2) in 3 seconds.


Phân tích khoa Chẩn đoán hình ảnh trang 9: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


ĐÃ LƯU!


In [5]:
df = pd.read_csv("ivie_article_crawling.csv")
df.shape

(1780, 3)

In [6]:
df.groupby("specialty").count()

,title,text
specialty,,
Bệnh hô hấp,61,61
Bệnh lý da liễu,19,19
Bệnh lý dị ứng,10,10
Bệnh lý giao mùa,15,15
Bệnh nội tiết,61,61
Bệnh thần kinh,13,13
Bệnh tim mạch,135,135
Bệnh tiêu hóa,72,72
Bệnh truyền nhiễm,88,88


# Cào thêm vì thất lạc

In [21]:
crawling("/nhi-khoa-0")

Selenium: Can't get connection to https://ivie.vn/nhi-khoa-0?page=1! Try again (1) in 3 seconds.


Phân tích khoa Nhi khoa trang 1: 100%|██████████| 19/19 [00:16<00:00,  1.14it/s]


ĐÃ LƯU!


Phân tích khoa Nhi khoa trang 2: 100%|██████████| 9/9 [00:03<00:00,  2.51it/s]


ĐÃ LƯU!


...

ĐÃ LƯU!


Phân tích khoa Nhi khoa trang 31: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]


ĐÃ LƯU!
Selenium: Can't get connection to https://ivie.vn/nhi-khoa-0?page=32! Try again (1) in 3 seconds.


Phân tích khoa Nhi khoa trang 32: 100%|██████████| 8/8 [00:05<00:00,  1.35it/s]


ĐÃ LƯU!
